[![Ireki Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uxue-sudupe/API-Eustat/blob/main/code_examples/eu/tutorial_R_eu.ipynb)

# **Nola erabili Eustaten APIa R-rekin**

[Eustaten datu bankuak](https://eu.eustat.eus/banku/indexArbol.html) Euskal Autonomia Erkidegoko gizarteari, ekonomiari eta biztanleriari buruzko datu ugari ditu.

Datuak publikoak dira eta modu desberdinetan eskura daitezke, besteak beste,  [hemen dokumentatutako](https://github.com/uxue-sudupe/API-Eustat/blob/main/README.md) APIaren bidez. APIak izugarri errazten du datuak eskuratzea eta berrerabiltzea. Hala ere, APIekin aurretik esperientziarik izan ezean, zaila izan daiteke ikertzaile, analista edo ikasle batentzat hura erabiltzea.

Tutorial honek gida azkar bat eskaintzen du Eustaten datuak APIaren bidez eskuratzeko. Datuak nola deskargatu, prozesatu eta dataset formatuko taulak nola sortu erakusten du.

Oso zabalduta dauden liburutegi batzuk erabili behar dira, `dplyr` edo `httr` adibidez, eta `rjstat` liburutegia, json-stat formatuan dauden datuak R-rekin tratatzeko espezifikoa dena.

In [ ]:
install.packages(c("httr", "jsonlite", "tidyr", "dplyr", "rjstat"), quiet = TRUE)

library(httr)
library(jsonlite)
library(tidyr)
library(dplyr)
library(rjstat)

# **1. Eustaten datuen katalogoa (taulen zerrenda) eskuratu**

Posible da Eustatek argitaratutako datu multzoen (taulen) zerrenda osoa ikustea eta jeistea, taula bakoitzaren identifikazio-kodea, izenburua eta azken eguneratze-data barne.

Eustaten datu bankuan dauden taulek osatzen dute katalogoa: <https://eu.eustat.eus/banku/id_All/indexArbol.html>

APIari egiten zaion eskaera batek hiru elementu nagusi izaten ditu: web helbide nagusia, funtziorako dei bat eta parametro multzo bat. Eustaten Datu Bankuko APIaren helbide nagusia honakoa da:

`https://www.eustat.eus/bankupx/api/v1/[lang]/DB`

Kasu honetan, APIari deia GET metodoaren bidez egiten zaio. Katalogoa lortzeko behar den parametro bakarra hizkuntza da (lang). Erantzuna JSON formatuan jasotzen da.

In [ ]:

# Eustaten datuen katalogoko sarbide puntua (endpoint)

DB <- GET("https://www.eustat.eus/bankupx/api/v1/eu/DB")
Katalogoa <- fromJSON(rawToChar(DB$content))
head(Katalogoa)


,id,type,text,updated
,<chr>,<chr>,<chr>,<chr>
1,PX__fe_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurralde eremuaren eta sexuaren arabera. 1997 - 2022",2023-01-09T15:04:41
2,PX__feinem_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurralde eremuaren eta sexuaren arabera",2021-02-17T10:04:22
3,PX__fepycl_pc02cm.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), Lurralde Historikoaren, indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2021-02-16T11:15:00
4,PX_0_fe_pc02cm.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), Lurralde Historikoaren, indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2022-02-15T10:31:14
5,PX_0_fe_pc02m.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2022-02-15T10:31:14
6,PX_0_fepycl_pc02cm.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), Lurralde Historikoaren, indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2021-09-14T08:41:11


**Hizkuntzak**

Zerrenda bera gaztelaniaz edo ingelesez ere eska daiteke. Horretarako, URLko hizkuntza parametroa aldatzea besterik ez da behar: `www.eustat.eus/bankupx/api/v1/[lang]/DB`

`[lang]` parametroa honako hiru aukeretako batekin ordezka daiteke: *eu* (euskera), *es* (gaztelania) edo *en* (ingelesa).

Adibidez, gaztelaniazko katalogoa eskuratzeko URL hau erabiliko litzateke:

In [ ]:

DB <- GET("https://www.eustat.eus/bankupx/api/v1/es/DB")
Catálogo <- fromJSON(rawToChar(DB$content))
head(Catálogo)


,id,type,text,updated
,<chr>,<chr>,<chr>,<chr>
1,PX__fe_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbitos territoriales y sexo. 1997 - 2022,2023-01-09T15:04:41
2,PX__feinem_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbitos territoriales y sexo,2021-02-17T10:04:22
3,PX__fepycl_pc02cm.px,t,Índice general de precios al consumo (IPC) por Territorio Histórico de la C.A. de Euskadi por índice y mes. Base 2016 (desde 2002),2021-02-16T11:15:00
4,PX_0_fe_pc02cm.px,t,Índice general de precios al consumo (IPC) por Territorio Histórico de la C.A. de Euskadi por índice y mes. Base 2016 (desde 2002),2022-02-15T10:31:14
5,PX_0_fe_pc02m.px,t,Índice general de precios al consumo (IPC) de la C.A. de Euskadi y del Estado por índice y mes. Base 2016 (desde 2002),2022-02-15T10:31:14
6,PX_0_fepycl_pc02cm.px,t,Índice general de precios al consumo (IPC) por Territorio Histórico de la C.A. de Euskadi por índice y mes. Base 2016 (desde 2002),2021-09-14T08:41:11


# **2. Taula baten metadatuak eskuratu**

Taula bateko metadatuek taularen izenburuari buruzko informazioa, taulan agertzen diren desagregazio-dimentsioak/aldagaiak eta hartzen dituzten balio edo kategoriei buruzko informazioa jasotzen dute. Metadatuetan kodeak eta testu deskribatzaileak datoz.

Taula jakin bateko metadatuetara sartzeko, interesatzen zaigun taularen `"id"` identifikazio kodea gehitu behar diogu datu bankuko url-ari. Helbideak egitura hau izango du:

`www.eustat.eus/bankupx/api/v1/[lang]/DB/[id]`

Adibidez, taula hau interesatzen bazaigu: "Euskal AEko biztanleria zenbatetsia, lurralde historikoaren, sexuaren eta jaioturtearen arabera", helbide honi egin behar diogu dei:

<https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010154_cepv1_ep17.px>

In [ ]:

metadatuak<- GET("https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010154_cepv1_ep17.px")
biztanleria_metadatuak <- fromJSON(rawToChar(metadatuak$content))
biztanleria_metadatuak

,code,text,values,valueTexts,time
,<chr>,<chr>,<list>,<list>,<lgl>
1,territorio histórico,lurralde historikoa,"00, 01, 48, 20","Euskal AE , Araba/Álava, Bizkaia , Gipuzkoa",NA
2,sexo,sexua,"10, 20, 30","Guztira , Gizonak , Emakumeak",NA
3,año de nacimiento,jaiotze urtea,"1000, 1109, 1110, 1120, 5800, 5810, 5820, 5830, 5840, 5850, 5860, 5870, 5880, 5890, 5900, 5910, 5920, 5930, 5940, 5950, 5960, 5970, 5980, 5990, 6000, 6010, 6020, 6030, 6040, 6050, 6060, 6070, 6080, 6090, 6100, 6110, 6120, 6130, 6140, 6150, 6160, 6170, 6180, 6190, 6200, 6210, 6220, 6230, 6240, 6250, 6260, 6270, 6280, 6290, 6300, 6310, 6320, 6330, 6340, 6350, 6360, 6370, 6380, 6390, 6400, 6410, 6420, 6430, 6440, 6450, 6460, 6470, 6480, 6490, 6500, 6510, 6520, 6530, 6540, 6550, 6560, 6570, 6580, 6590, 6600, 6610, 6620, 6630, 6640, 6650, 6660, 6670, 6680, 6690, 6700, 6710, 6720, 6730, 6740, 6750, 6760, 6770, 6780, 6790, 6800, 6810, 6820, 6830, 6840, 6850, 6860, 6870, 6880, 6890, 6900, 6910, 6920, 6930, 6940, 6950, 6960, 6970, 6980, 6990, 7000, 7010, 7020, 7030, 7040, 7050, 7060, 7070, 7080, 7090, 7100, 7110, 7120, 7130, 7140, 7150, 7160, 7170, 7180, 7190, 7200, 7210, 7220, 7230, 7240, 7250","Guztira, 2023 , 2022 , 2021 , 2020 , 2019 , 2018 , 2017 , 2016 , 2015 , 2014 , 2013 , 2012 , 2011 , 2010 , 2009 , 2008 , 2007 , 2006 , 2005 , 2004 , 2003 , 2002 , 2001 , 2000 , 1999 , 1998 , 1997 , 1996 , 1995 , 1994 , 1993 , 1992 , 1991 , 1990 , 1989 , 1988 , 1987 , 1986 , 1985 , 1984 , 1983 , 1982 , 1981 , 1980 , 1979 , 1978 , 1977 , 1976 , 1975 , 1974 , 1973 , 1972 , 1971 , 1970 , 1969 , 1968 , 1967 , 1966 , 1965 , 1964 , 1963 , 1962 , 1961 , 1960 , 1959 , 1958 , 1957 , 1956 , 1955 , 1954 , 1953 , 1952 , 1951 , 1950 , 1949 , 1948 , 1947 , 1946 , 1945 , 1944 , 1943 , 1942 , 1941 , 1940 , 1939 , 1938 , 1937 , 1936 , 1935 , 1934 , 1933 , 1932 , 1931 , 1930 , 1929 , 1928 , 1927 , 1926 , 1925 , 1924 , 1923 , 1922 , 1921 , 1920 , 1919 , 1918 , 1917 , 1916 , 1915 , 1914 , 1913 , 1912 , 1911 , 1910 , 1909 , 1908 , 1907 , 1906 , 1905 , 1904 , 1903 , 1902 , 1901 , 1900 , 1899 , 1898 , 1897 , 1896 , 1895 , 1894 , 1893 , 1892 , 1891 , 1890 , 1889 , 1888 , 1887 , 1886 , 1885 , 1884 , 1883 , 1882 , 1881 , 1880 , 1879 , 1878 , 1877 , 1876 , <= 1875",NA
4,periodo,aldia,"19760101, 19770101, 19780101, 19790101, 19800101, 19810101, 19820101, 19830101, 19840101, 19850101, 19860101, 19870101, 19880101, 19890101, 19900101, 19910101, 19920101, 19930101, 19940101, 19950101, 19960101, 19970101, 19980101, 19990101, 20000101, 20010101, 20020101, 20030101, 20040101, 20050101, 20060101, 20070101, 20080101, 20090101, 20100101, 20110101, 20120101, 20130101, 20140101, 20150101, 20160101, 20170101, 20180101, 20190101, 20200101, 20210101, 20220101, 20220701, 20230101, 20230701, 20240101","1976/01/01, 1977/01/01, 1978/01/01, 1979/01/01, 1980/01/01, 1981/01/01, 1982/01/01, 1983/01/01, 1984/01/01, 1985/01/01, 1986/01/01, 1987/01/01, 1988/01/01, 1989/01/01, 1990/01/01, 1991/01/01, 1992/01/01, 1993/01/01, 1994/01/01, 1995/01/01, 1996/01/01, 1997/01/01, 1998/01/01, 1999/01/01, 2000/01/01, 2001/01/01, 2002/01/01, 2003/01/01, 2004/01/01, 2005/01/01, 2006/01/01, 2007/01/01, 2008/01/01, 2009/01/01, 2010/01/01, 2011/01/01, 2012/01/01, 2013/01/01, 2014/01/01, 2015/01/01, 2016/01/01, 2017/01/01, 2018/01/01, 2019/01/01, 2020/01/01, 2021/01/01, 2022/01/01, 2022/07/01, 2023/01/01, 2023/07/01, 2024/01/01",TRUE


Metadatuek taularen izenburua dute, aldagaien eta balioen kodeekin eta literalekin batera. Baliagarria izan daiteke aldagaiak beren balioekin batera erakusten dituen taula bat sortzea, kodeen eta etiketa deskribatzaileen arteko korrespondentzia erakusten dituena.

In [ ]:
aldagaiak <- biztanleria_metadatuak$variables

biztanleria_aldagaiak <- aldagaiak %>%
  mutate(
    values = lapply(values, unlist),
    valueTexts = lapply(valueTexts, unlist)
  ) %>%
  unnest(c(values, valueTexts))

biztanleria_aldagaiak

code,text,values,valueTexts,time
<chr>,<chr>,<chr>,<chr>,<lgl>
territorio histórico,lurralde historikoa,00,Euskal AE,NA
territorio histórico,lurralde historikoa,01,Araba/Álava,NA
territorio histórico,lurralde historikoa,48,Bizkaia,NA
territorio histórico,lurralde historikoa,20,Gipuzkoa,NA
sexo,sexua,10,Guztira,NA
sexo,sexua,20,Gizonak,NA
sexo,sexua,30,Emakumeak,NA
año de nacimiento,jaiotze urtea,1000,Guztira,NA
año de nacimiento,jaiotze urtea,1109,2023,NA


# **3. Taula bateko datuak eskuratu**

**Taula bateko datu guztiak eskuratzeko kontsulta**

Tauletako datuak eskuratzeko, kontsulta bat egin behar dugu aurreko puntuan ikusi dugun helbide berean, POST funtzioarekin.

rjstat izeneko modulua erabiliko dugu json-stat formatuan deskargatuko ditugun datuekin lan egiteko. Adibide honetan, "Euskal AEko barne produktu gordina (BPG-eskaintza), lurralde historiko, jarduera-adar (A-38), datu-mota eta neurri-motaren arabera" taularen datu guztiak kontsultatuko ditugu.

In [ ]:

# Taulako sarbide puntua (endpoint)

url <- "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_170112_cpib_pib01d.px"

# Eskaeraren edukia: filtrorik gabe (taula osoa). Emaitza JSON-stat v.1.2 formatuan lortzen da, besterik esaten ez bada
kontsulta <- '{"query":[]}'

d.tmp <- POST(url , body = kontsulta, encode = "json", verbose())
taula <- fromJSONstat(content(d.tmp, "text"), naming = "label", use_factors = FALSE, silent = FALSE)
BPG <- taula[[1]]
head(BPG)

,lurralde historikoa,sektorea,datu-mota,neurri-mota,aldia,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1995,449513
2,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1996,439283
3,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1997,466735
4,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1998,564307
5,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1999,605399
6,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),2000,514139


**Taula bateko datuak eskatu, kodeekin**

Baliteke datu horiek aldagaien kodeekin eskuratu nahi izatea, eta ez etiketa deskribatzaileekin edo literalekin. Kasu horretan, izenak kode gisa `(naming = "id")` jeisteko eskatuko dugu, eta ez testu gisa `(naming = "label")`.

In [ ]:
taula<- fromJSONstat(content(d.tmp, "text"), naming = "id", use_factors = FALSE, silent = FALSE)
BPG_kodeekin <- taula[[1]]  # Taularen aurrenengo zerrenda bakarrik interesatzen zaigu
head(BPG_kodeekin)

,territorio histórico,sector,tipo de dato,tipo de medida,periodo,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,00,01,10,10,1995,449513
2,00,01,10,10,1996,439283
3,00,01,10,10,1997,466735
4,00,01,10,10,1998,564307
5,00,01,10,10,1999,605399
6,00,01,10,10,2000,514139


**Taula bateko datuen selekzio bat eskuratu**

Datu gehiegi eskatzea, gero erdia bota behar izateko, espazioa eta baliabideak alferrik galtzea da. Horregatik, taularen datuen parte bat bakarrik interesatzen zaigunean, eta ez taularen eduki osoa, kontsulta bat egin behar dugu intereseko aldagaiak eta balioak hautatuta.


Kontsulta definitzeko, aldagaien izena/kodea eta balioak ezagutu behar dira. Aurreko puntuan ikusi dugu nola lortu taula baten metadatuak.


Baina urrats hori errazteko, Eustaten webguneak berak aldagaiak eta balioak hautatzeko interfaze bat du taula bakoitzarentzat. Interfaze horretan, erraz aukeratu ahal izango ditugu interesatzen zaizkigun datuak, eta JSON formatuko kontsulta lortu.

Adibidez, sexuaren araberako langabezia-tasaren datuak nahi izanez gero, datu bankuko taula honetako aldagaiak hautatzen diren orrira joango ginateke:

<https://www.eustat.eus/bankupx/pxweb/eu/DB/-/PX_050403_cpra_tab01.px>

Eta interesatzen zaigun hautaketa egingo dugu: Euskal AEko langabezia-tasaren datuak, 2 sexuak (eta guztizkoa) eta azken 3 urteetako hiruhilekoko datuak hautatuko ditugu.

***Hautaketaren adibidea***

![](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Langabezia_aukeraketa.png)

Aukeraketa egin ondoren, `"Jarraitu"` eta `"Taula hau eskuragarri egin zure aplikazioan"` sakatuko dugu. Atal honetan, eskaera egin behar zaion URLa eta json kontsulta agertuko dira. Kontsulta hori gure kodean sartuko dugu.

***Kontsultaren adibidea***

![](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Langabezia_kontsulta.png)

In [ ]:
url <- "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_050403_cpra_tab01.px"
kontsulta <- '{"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
        {"code": "territorio histórico","selection": {"filter": "item", "values": [ "00"]}},
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50" ]}},
        {"code": "periodo","selection": {"filter": "item","values": ["2023", "2024","2025" ]}}],
         "response": {"format": "json-stat"}} '

d.tmp <- POST(url , body = kontsulta, encode = "json", verbose())
taula <- fromJSONstat(content(d.tmp, "text"), naming = "label", use_factors = FALSE, silent = FALSE)
Langabezia <- taula[[1]]
head(Langabezia)

,tasa (%),lurralde historikoa,sexua,hiruhilekoa,aldia,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2023,8.9
2,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2024,7.9
3,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2025,7.0
4,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2023,7.6
5,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2024,7.5
6,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2025,NA


**Aldaketak kontsultan**

`Filter: item` egiten denean aldagai baten gainean, filtratu nahi diren balioak hautatzen dira (bat edo gehiago).

Balio guztiak hautatzen badira, ez da aldagai horren gainean filtratzen, eta ez da kontsultan agertuko. Efektu bera du "lurralde historikoa" aldagaiaren kode-lerroarekin orain egingo dugun bezala `{"code":"variable"....}` kodearekin hasten den lerroa ezabatzeak ere.

`Top` filtroa ere erabil dezakegu, aldagaiaren azken balioak bakarrik hautatzeko. Top filtroa batez ere "aldia", "urtea" edo antzeko aldagaiekin erabiltzen da.

In [ ]:

url = "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_050403_cpra_tab01.px"

kontsulta = '{"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50" ]}},
        {"code": "periodo","selection": {"filter": "top","values": ["2"]}}],
         "response": {"format": "json-stat"}}'

d.tmp <- POST(url , body = kontsulta, encode = "json", verbose())
taula <- fromJSONstat(content(d.tmp, "text"), naming = "label", use_factors = FALSE, silent = FALSE)
Langabezia <- taula[[1]]
head(Langabezia)


,tasa (%),lurralde historikoa,sexua,hiruhilekoa,aldia,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2024,7.9
2,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2025,7.0
3,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2024,7.5
4,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2025,NA
5,Langabezi tasa,Euskal AE,Guztira,3. Hiruhilekoa,2024,7.2
6,Langabezi tasa,Euskal AE,Guztira,3. Hiruhilekoa,2025,NA


# **Irteerako beste formatu batzuk**

Tauletako datuetarako irteera lehenetsia JSON-stat da (1.2 bertsioa). JSON formatua da ohikoena APIetan, datuen tratamendu malgua ahalbidetzen duelako, script-etan (R, Python, etab.) automatikoki prozesatzeko egokia delako, eta CSV bezalako artxibo lauetan aurkez daitezkeen anbiguotasunak saihesten dituelako.

APIari eskatutako datuak CSV eta excel formatuetan ere deskarga daitezke. Irteera horiek egokiagoak izan daitezke erabiltzaile ez hain teknikoentzat, sarrera gisa csv edo excel behar diren prozesuentzat (csv/excel bidez dashboard-ak sortzeko adibidez) edo helburua analisia ez denean, baizik eta APIak ematen dituen datuen deskarga erraztea publikoari.

Jarraian csv (3 mota) eta excel irteerak nola eskatu eta zein itxura duten azaltzen da. Irteera bakoitzaren kopia bat gordetzen da.

In [ ]:
# Irteera CSV - testu laua

url <- "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
kontsulta <- '{"query": [], "response": {"format": "csv"}}'

erantzuna_csv <- POST(url, body = kontsulta)
writeBin(content(erantzuna_csv, "raw"), "ama_hizkuntza.csv")


In [ ]:
#  CSV2 moduko irteera - "pivot-friendly" formatua, taula dinamikoentzat egokia

url <- "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
kontsulta <- '{"query": [], "response": {"format": "csv2"}}'

erantzuna_csv2 <- POST(url, body = kontsulta, encode = "raw", content_type_json())

writeBin(content(erantzuna_csv2, "raw"), "ama_hizkuntza_csv2.csv")


In [ ]:
# CSV3 moduko irteera - CSV2 bezala, baina kodeekin testua izan beharrean

url <- "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
kontsulta <- '{"query": [], "response": {"format": "csv3"}}'

erantzuna_csv3 <- POST(url, body = kontsulta, encode = "raw", content_type_json())

writeBin(content(erantzuna_csv3, "raw"), "ama_hizkuntza_csv3.csv")


In [ ]:
# Excel irteera

url <- "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
kontsulta <- '{"query": [], "response": {"format": "xlsx"}}'

erantzuna_xlsx <- POST(url, body = kontsulta, encode = "raw", content_type_json())

writeBin(content(erantzuna_xlsx, "raw"), "ama_hizkuntza.xlsx")
